# Regional Wind Speed


In [1]:
%load_ext autotime

time: 195 µs (started: 2021-07-25 15:02:42 -03:00)


In [2]:
import sys
sys.path.append('..')

time: 271 µs (started: 2021-07-25 15:02:42 -03:00)


In [3]:
import pandas as pd
import numpy as np
import xarray as xr
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as st


from cycling_digital_twin.functions import load_route, append_deltas

time: 1.45 s (started: 2021-07-25 15:02:42 -03:00)


In [4]:
gpx_path = '../data/record_route.gpx'
DATA_PATH = '../data/historical_wind_data.nc'

time: 217 µs (started: 2021-07-25 15:02:44 -03:00)


In [5]:
def estimated_time(distance,
                   uphill,
                   downhill,
                   wind_speed=0,
                   flat_speed=26.5,
                   alpha=0.02,
                   beta=-1/150,
                   gamma=0.6):
    time = (distance + alpha * uphill + beta * downhill)
    time /= (flat_speed + gamma * wind_speed)
    return time


route_df = (load_route(gpx_path).pipe(append_deltas)
                                .assign(distance_in_km=lambda df: df.delta.cumsum() / 1000)
                                .assign(delta_ele=lambda df: df.ele.diff())
                                .dropna()
                                .assign(cumm_uphill=lambda df: df.delta_ele
                                                                 .apply(lambda x: max(x, 0))
                                                                 .cumsum())
                                .assign(cumm_downhill=lambda df: df.delta_ele
                                                                   .apply(lambda x: min(x, 0))
                                                                   .abs()
                                                                   .cumsum())
                                .assign(est_time=lambda df: estimated_time(df.distance_in_km,
                                                                           df.cumm_uphill,
                                                                           df.cumm_downhill))
           )

Calculating deltas:   0%|          | 0/20839 [00:00<?, ?it/s]

time: 2.39 s (started: 2021-07-25 15:02:44 -03:00)


In [6]:
route_df['est_day'] = pd.cut(route_df.est_time, 28, labels=False) + 1

time: 1.92 ms (started: 2021-07-25 15:02:46 -03:00)


In [8]:
ds = xr.open_dataset(DATA_PATH)
mask = (ds.time.dt.month == 9)
ds = ds.isel(time=mask)
mask_2 = ds.lon < -52
ds = ds.isel(lon=mask_2)
ds = ds.assign(speed_in_kph=3.6 * np.sqrt(ds.U2M * ds.U2M + ds.V2M * ds.V2M))
ds = ds.assign(angle_in_degree=np.arctan2(ds.U2M, ds.V2M) * 360 / (2 * np.pi))



time: 25.7 s (started: 2021-07-25 15:03:24 -03:00)


In [9]:
import geocoder

time: 106 ms (started: 2021-07-25 15:03:50 -03:00)


In [12]:
MESO_REGIONS = {'amazon_1': ('Uiramutã / RR, Brazil', 'Manaus / AM, Brazil'),
                'amazon_2': ('Manaus / AM, Brazil', 'Humaitá / AM, Brazil'),
                'rondonia_1': ('Humaitá / AM, Brazil', 'Vilhena / RO, Brazil'),
                'rondonia_2': ('Vilhena / RO, Brazil', 'Cuiabá / MT, Brazil'),
                'cerrado_1': ('Cuiabá / MT, Brazil', 'São Gabriel do Oeste / MS, Brazil'),
                'cerrado_2': ('São Gabriel do Oeste / MS, Brazil', 'Guairá / PR, Brazil'),
                'parana_1': ('Guairá / PR, Brazil', 'Palmeira das Missões / RS, Brazil'),
                'parana_2': ('Palmeira das Missões / RS, Brazil', 'Chuí / RS, Brazil')
               }

for label, tup in MESO_REGIONS.items():
    p1 = geocoder.osm(tup[0]).latlng
    p2 = geocoder.osm(tup[1]).latlng

    p = [(p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2, ]

    a = (ds.sel(lat=p[0],lon=p[1], method='nearest')
           .assign(hour=lambda ds: ds.time.dt.hour))
    
    x = a.speed_in_kph
    y = a.angle_in_degree.values
    
    print(f"Stats for {label}")
    print(f"Mean Wind Speed (km/h): {np.median(x)}")
    print(f"Wind Speed 90% IQR (km/h): {np.percentile(x, 95) - np.percentile(a.speed_in_kph, 5)}")
    print(f"Mean Wind Angle (deg): {st.circmean(y[x > 1], low=-180, high=180)}")
    print(f"Std Wind Angle (deg): {st.circstd(y[x > 1], low=-180, high=180)}")
    print("-----")

Stats for amazon_1
Mean Wind Speed (km/h): 0.018268730491399765
Wind Speed 90% IQR (km/h): 1.8345785915851593
Mean Wind Angle (deg): -158.24173913472362
Std Wind Angle (deg): 91.10543070429644
-----
Stats for amazon_2
Mean Wind Speed (km/h): 6.786664009094238
Wind Speed 90% IQR (km/h): 13.785523390769955
Mean Wind Angle (deg): -106.27004768343663
Std Wind Angle (deg): 85.77119866592167
-----
Stats for rondonia_1
Mean Wind Speed (km/h): 0.9865562915802002
Wind Speed 90% IQR (km/h): 2.28325600028038
Mean Wind Angle (deg): -105.85862440859175
Std Wind Angle (deg): 116.84194464675204
-----
Stats for rondonia_2
Mean Wind Speed (km/h): 5.7807536125183105
Wind Speed 90% IQR (km/h): 14.260575520992276
Mean Wind Angle (deg): -90.03636142266298
Std Wind Angle (deg): 89.95379545154806
-----
Stats for cerrado_1
Mean Wind Speed (km/h): 8.09202766418457
Wind Speed 90% IQR (km/h): 13.861359226703641
Mean Wind Angle (deg): -103.58598141510755
Std Wind Angle (deg): 89.63306409853612
-----
Stats for cer

In [ ]:
 a